In [1]:
!pip install transformers datasets pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv('/content/ielts_writing_dataset (1).csv')
df.head()

,Task_Type,Question,Essay,Examiner_Commen,Task_Response,Coherence_Cohesion,Lexical_Resource,Range_Accuracy,Overall
0,1,The bar chart below describes some changes abo...,"Between 1995 and 2010, a study was conducted r...",NaN,NaN,NaN,NaN,NaN,5.5
1,2,Rich countries often give money to poorer coun...,Poverty represents a worldwide crisis. It is t...,NaN,NaN,NaN,NaN,NaN,6.5
2,1,The bar chart below describes some changes abo...,The left chart shows the population change hap...,NaN,NaN,NaN,NaN,NaN,5.0
3,2,Rich countries often give money to poorer coun...,Human beings are facing many challenges nowada...,NaN,NaN,NaN,NaN,NaN,5.5
4,1,The graph below shows the number of overseas v...,Information about the thousands of visits from...,NaN,NaN,NaN,NaN,NaN,7.0


In [3]:
df = df[["Essay", "Overall"]]
df.head()

,Essay,Overall
0,"Between 1995 and 2010, a study was conducted r...",5.5
1,Poverty represents a worldwide crisis. It is t...,6.5
2,The left chart shows the population change hap...,5.0
3,Human beings are facing many challenges nowada...,5.5
4,Information about the thousands of visits from...,7.0


In [4]:
df_test = df.sample(n = 100, random_state = 42)
df_test.head()

,Essay,Overall
752,The graphic illustrates the nutritional consis...,6.5
857,Life is full of competition. Soon after the re...,7.5
629,Sugar-based drinks has become a popular bevera...,7.0
1411,"In the Third World, children are usually sent ...",6.0
974,The line graph illustrates the proportion of m...,5.5


In [5]:
df = df[~df.index.isin(df_test.index)]
df.shape

(1335, 2)

In [6]:
dataset = Dataset.from_pandas(df, preserve_index = False)
dataset = dataset.train_test_split(test_size=0.1)

In [7]:
import transformers
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=1)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Essay', 'Overall'],
        num_rows: 1201
    })
    test: Dataset({
        features: ['Essay', 'Overall'],
        num_rows: 134
    })
})

In [9]:
def tokenize_function(examples):
    label = examples["Overall"]
    examples = tokenizer(examples["Essay"], truncation=True, padding="max_length", max_length=256)

    examples["label"] = float(label)
    return examples

for split in dataset:
    dataset[split] = dataset[split].map(tokenize_function, remove_columns=["Essay", "Overall"])

Map:   0%|          | 0/1201 [00:00<?, ? examples/s]

Map:   0%|          | 0/134 [00:00<?, ? examples/s]

In [24]:
#tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'Overall'])

In [10]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.7 MB/s eta 0:00:00


In [11]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)

    mse = mean_squared_error(labels, logits)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)
    single_squared_errors = ((logits - labels).flatten()**2).tolist()
    accuracy = sum([1 for e in single_squared_errors if e < 0.25]) / len(single_squared_errors)

    return {"mse": mse, "mae": mae, "r2": r2, "accuracy": accuracy}

In [12]:
from transformers import TrainingArguments, Trainer, TrainerCallback
import torch

training_args = TrainingArguments(output_dir="./fine_tuned_bert",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  learning_rate= 8e-5,
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32,
                                  gradient_accumulation_steps = 8,
                                  num_train_epochs=20,
                                  save_total_limit = 2,
                                  save_strategy = 'epoch',
                                  lr_scheduler_type = "linear",
                                  optim = "adamw_torch",
                                  weight_decay = 0.01,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="mae",
                                  warmup_steps=500,
                                  )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics_for_regression
)

class MetricsCallback(TrainerCallback):
    def __init__(self):
        self.training_history = {"train": [], "eval": []}

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            if "loss" in logs:  # Training logs
                self.training_history["train"].append(logs)
            elif "eval_loss" in logs:  # Evaluation logs
                self.training_history["eval"].append(logs)

metrics_callback = MetricsCallback()
trainer.add_callback(metrics_callback)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: edbertw (edbertw-migrasia) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Mse,Mae,R2,Accuracy
1,42.078400,33.887280,33.887280,5.733872,-32.186348,0.000000
2,11.105600,1.040224,1.040224,0.794654,-0.018708,0.380597
3,1.176500,0.943572,0.943572,0.773422,0.075945,0.410448
4,0.872200,0.625585,0.625584,0.634337,0.387355,0.470149
5,0.716700,0.519421,0.519421,0.580184,0.491323,0.522388
6,0.528600,1.104981,1.104981,0.887199,-0.082125,0.276119
7,0.453100,0.706095,0.706095,0.664853,0.308510,0.462687
8,0.389900,1.041111,1.041111,0.860139,-0.019576,0.305970
9,0.378700,0.519705,0.519705,0.586704,0.491045,0.514925
10,0.306000,1.107717,1.107717,0.881921,-0.084804,0.291045


TrainOutput(global_step=760, training_loss=2.9876057740889097, metrics={'train_runtime': 1297.94, 'train_samples_per_second': 18.506, 'train_steps_per_second': 0.586, 'total_flos': 3159935402833920.0, 'train_loss': 2.9876057740889097, 'epoch': 20.0})

In [14]:
dataset_2 = Dataset.from_pandas(df_test, preserve_index = False)
dataset_2 = dataset_2.map(tokenize_function, remove_columns=["Essay", "Overall"])

trainer.eval_dataset=dataset_2
trainer.evaluate()


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

{'eval_loss': 34.489253997802734,
 'eval_mse': 34.489253997802734,
 'eval_mae': 5.771015167236328,
 'eval_r2': -28.104854583740234,
 'eval_accuracy': 0.0,
 'eval_runtime': 1.4397,
 'eval_samples_per_second': 69.457,
 'eval_steps_per_second': 2.778,
 'epoch': 20.0}